# Allocation Algorithm

In [5]:
import pandas as pd

## Data Importing

In [6]:
#Import file
file_path = '2024-02-20_Case_Granular_Energy.xlsx'

In [7]:
# Load consumption & generation data from different portfolios and format it 
abc_demand = pd.read_excel(file_path,
    sheet_name='ABC',
    header=0,  
    usecols="A:C", 
    parse_dates=['Period']
    ).fillna(0).round()
abc_demand['Portfolio'] = 'ABC'


xyz_demand = pd.read_excel(file_path,
    sheet_name='XYZ',
    header=0,  
    usecols="A:C", 
    parse_dates=['Period']
    ).fillna(0).round()
xyz_demand['Portfolio'] = 'XYZ'

super_green_demand = pd.read_excel(file_path,
    sheet_name='Super Green',
    header=0,
    usecols="A:C",
    parse_dates=['Period']  
    ).fillna(0).round()
super_green_demand['Portfolio'] = 'Super Green'


generation_data = pd.read_excel(file_path,  
    sheet_name='Generation',
    header=0,
    usecols="A:D",
    parse_dates=['Generation month']  
    ).fillna(0).round()



In [8]:
#Concatenate the entire demand into one data frame
combined_demand = pd.concat([abc_demand, xyz_demand, super_green_demand], ignore_index=True)  
combined_demand.shape

(1826, 4)

In [9]:
#Make sure the columns are in the right formate 
combined_demand['Consumption'] = pd.to_numeric(combined_demand['Consumption'], errors='coerce').fillna(0)
generation_data['Quantity'] = pd.to_numeric(generation_data['Quantity'], errors='coerce').fillna(0)


In [46]:
# Make sure the column names are stripped
generation_data['Technology'] = generation_data['Technology'].str.strip()
generation_data['Technology'].value_counts()

Technology
Photovoltaic      509
Landfill Gas      101
Wind               84
Off-shore wind     78
Hydro              34
Biomass            24
Biogas             16
Sewage gas         10
Name: count, dtype: int64

In [71]:
generation_data

,Generation month,plant,Technology,Quantity
0,2022-04-01,plant 207,Photovoltaic,2000
1,2022-04-01,plant 247,Hydro,17000
2,2022-04-01,plant 21,Photovoltaic,18000
3,2022-04-01,plant 276,Wind,50000
4,2022-04-01,plant 108,Photovoltaic,6000
...,...,...,...,...
851,2023-03-01,plant 253,Off-shore wind,81000
852,2023-03-01,plant 219,Off-shore wind,5000
853,2023-03-01,plant 93,Photovoltaic,21000
854,2023-03-01,plant 141,Photovoltaic,3000


## Perform Allocation Algorithm

In [99]:

def refine_allocation(combined_demand, generation_data):
    allocations = []
    allocation_id = 1

    # Tech preferences based on portfolio
    tech_preferences = {
        'XYZ': ['Photovoltaic', 'Wind'],
        'Super Green': ['Wind', 'Photovoltaic', 'Hydro'],
        'ABC': ['Wind', 'Hydro', 'Biogas', 'Biomass', 'Landfill Gas', 'Off-shore wind', 'Photovoltaic', 'Sewage gas']
    }

    # Convert Period and Generation month to datetime format if not already
    combined_demand['Period'] = pd.to_datetime(combined_demand['Period'])
    generation_data['Generation month'] = pd.to_datetime(generation_data['Generation month'])

    for portfolio, preferences in tech_preferences.items():
        # Filter demand for the portfolio
        portfolio_demand = combined_demand[combined_demand['Portfolio'] == portfolio]
        
        for month in portfolio_demand['Period'].unique():
            month_demand = portfolio_demand[portfolio_demand['Period'] == month]
            
            for tech in preferences:
                # Filter generation data for the month and technology
                month_gen_data = generation_data[(generation_data['Technology'] == tech) & (generation_data['Generation month'] == month)]
                
                for _, gen_row in month_gen_data.iterrows():
                    plant = gen_row['plant']
                    available_quantity = gen_row['Quantity']
                    
                    for _, demand_row in month_demand.iterrows():
                        consumer_name = demand_row['Consumer Name']
                        consumer_demand = demand_row['Consumption']
                        
                        # Calculate allocated quantity
                        allocated_quantity = min(available_quantity, consumer_demand)
                        if allocated_quantity > 0:
                            # Append allocation
                            allocations.append({
                                'Allocation ID': allocation_id,
                                'Month': month.strftime('%Y-%m'),
                                'Portfolio': portfolio,
                                'Consumer': consumer_name,
                                'Plant': plant,
                                'Technology': tech,
                                'Quantity': allocated_quantity
                            })
                            allocation_id += 1
                            # Update available quantity
                            available_quantity -= allocated_quantity
                            # Update demand in month_demand DataFrame
                            month_demand.loc[month_demand['Consumer Name'] == consumer_name, 'Consumption'] -= allocated_quantity
                        
                        # Break if plant's available quantity is depleted
                        if available_quantity <= 0:
                            break

    allocations_df = pd.DataFrame(allocations)
    return allocations_df


In [108]:
#View Output & 
allocation = refine_allocation(combined_demand, generation_data)
allocation.head(20)

,Allocation ID,Month,Portfolio,Consumer,Plant,Technology,Quantity
0,1,2022-05,XYZ,Customer16,plant 235,Photovoltaic,20.0
1,2,2022-06,XYZ,Customer16,plant 114,Photovoltaic,19.0
2,3,2022-07,XYZ,Customer16,plant 88,Photovoltaic,20.0
3,4,2022-08,XYZ,Customer16,plant 374,Photovoltaic,22.0
4,5,2022-09,XYZ,Customer17,plant 205,Photovoltaic,1374.0
5,6,2022-09,XYZ,Customer17,plant 205,Photovoltaic,84.0
6,7,2022-09,XYZ,Customer17,plant 205,Photovoltaic,2918.0
7,8,2022-09,XYZ,Customer17,plant 205,Photovoltaic,364.0
8,9,2022-09,XYZ,Customer17,plant 205,Photovoltaic,4874.0
9,10,2022-09,XYZ,Customer17,plant 205,Photovoltaic,109.0


## Conduct QA

In [106]:
# Test and QA results by playing around with some of the consumers
test_consumer = allocation[allocation['Consumer']=='Customer21']
test_sum = temp['Quantity'].sum()
test_consumer

,Allocation ID,Month,Portfolio,Consumer,Plant,Technology,Quantity
25,26,2022-10,XYZ,Customer21,plant 264,Photovoltaic,595.0
26,27,2022-10,XYZ,Customer21,plant 264,Photovoltaic,520.0
175,176,2022-04,Super Green,Customer21,plant 276,Wind,50000.0
176,177,2022-04,Super Green,Customer21,plant 90,Wind,54000.0
177,178,2022-04,Super Green,Customer21,plant 389,Wind,17000.0
...,...,...,...,...,...,...,...
316,317,2022-08,Super Green,Customer21,plant 380,Photovoltaic,2482.0
333,334,2022-09,Super Green,Customer21,plant 33,Wind,9000.0
334,335,2022-09,Super Green,Customer21,plant 307,Wind,24000.0
335,336,2022-09,Super Green,Customer21,plant 315,Wind,36000.0


## Write Allocation Results to Excel

!pip install openpyxl
!pip install --upgrade pandas openpyxl

In [114]:
# Add allocation table and unmatched capacities to excel file 
with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='new') as writer:
    allocation.to_excel(writer, sheet_name='Allocations', index=False)

## Final Check 

In [115]:
result_check = pd.read_excel(file_path,
    sheet_name='Allocations').groupby('Portfolio')
result_check.head()

,Allocation ID,Month,Portfolio,Consumer,Plant,Technology,Quantity
0,1,2022-05,XYZ,Customer16,plant 235,Photovoltaic,20.0
1,2,2022-06,XYZ,Customer16,plant 114,Photovoltaic,19.0
2,3,2022-07,XYZ,Customer16,plant 88,Photovoltaic,20.0
3,4,2022-08,XYZ,Customer16,plant 374,Photovoltaic,22.0
4,5,2022-09,XYZ,Customer17,plant 205,Photovoltaic,1374.0
175,176,2022-04,Super Green,Customer21,plant 276,Wind,50000.0
176,177,2022-04,Super Green,Customer21,plant 90,Wind,54000.0
177,178,2022-04,Super Green,Customer21,plant 389,Wind,17000.0
178,179,2022-04,Super Green,Customer21,plant 33,Wind,2000.0
179,180,2022-04,Super Green,Customer21,plant 318,Wind,62000.0
